# Titanic

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn.metrics import mean_squared_error, r2_score

/home/ec2-user/miniconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [38]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [85]:
# Sets
firstSet = train_df.drop(['Name', 'Ticket', 'Cabin', 'Parch', 'Fare', 'Embarked', 'PassengerId', 'Age'],axis=1)
firstSet.dropna(inplace=True)

#Identify X - Segregate X
X = firstSet.drop(['Survived'],axis=1)
X['Sex'].replace('male',0, inplace=True)
X['Sex'].replace('female',1, inplace=True)
X.dropna(inplace=True)

#Identify Y - Segregate Y
Y = firstSet['Survived']

#Create a linear regression object - Not correct for this dataset. But I try to fit one for learning purposes
lmod = lm.LinearRegression()
lmod.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [86]:
#Print parameters
print('Intercept : ',lmod.intercept_)
pd.DataFrame(list(zip(X.columns, lmod.coef_)), columns=['features','estimatedCoeffs'])

Intercept :  0.5702787374067556


,features,estimatedCoeffs
0,Pclass,-0.153811
1,Sex,0.526117
2,SibSp,-0.032039


In [87]:
# Get the fitted values
y_fittedVals = lmod.predict(X)

# Get the MSE
print("Mean squared error: %.2f"
      % mean_squared_error(Y, y_fittedVals))

print('R-Square: %.2f' % r2_score(Y, y_fittedVals))

#y_fittedVals

Mean squared error: 0.15
R-Square: 0.37


## Doing the same using Stats model
[Link](http://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html)

In [88]:
import statsmodels.api as sm

In [89]:
# add_constant has to be done by the user. Otherwise it will not add a intercept to the model
X2 = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:               Survived   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     175.8
Date:                Tue, 06 Nov 2018   Prob (F-statistic):           2.01e-89
Time:                        05:47:41   Log-Likelihood:                -414.12
No. Observations:                 891   AIC:                             836.2
Df Residuals:                     887   BIC:                             855.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5703      0.041     14.030      0.0

In [90]:
X2.head()

,const,Pclass,Sex,SibSp
0,1.0,3,0,1
1,1.0,1,1,1
2,1.0,3,1,0
3,1.0,1,1,1
4,1.0,3,0,0


## Logistic Regression
Above Linear Regression model is only for representation sake. We need to use Logistic regression for this activity

In [91]:
logit = sm.Logit(Y, X)
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.464304
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  891
Model:                          Logit   Df Residuals:                      888
Method:                           MLE   Df Model:                            2
Date:                Tue, 06 Nov 2018   Pseudo R-squ.:                  0.3028
Time:                        05:47:45   Log-Likelihood:                -413.69
converged:                       True   LL-Null:                       -593.33
                                        LLR p-value:                 9.690e-79
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Pclass        -0.6773      0.050    -13.615      0.000      -0.775      -0.580
Sex            2.7787      0.183     15.145      0.000       2.419       3.138
SibSp         -0.2302      0.091     -2.519      0.012      -0.409      -0.051
==============================================================================
"""

In [92]:
#test_df['Age'].isnull() & (test_df['Name'].str.contains('Master'))]#.replace('NaN',5,inplace=True)

test_df['Age'].isnull().any()

True

In [95]:
# Prepare test set
X_test = test_df.drop(['Name', 'Ticket', 'Cabin', 'Parch', 'Fare', 'Embarked', 'PassengerId','Age'],axis=1)
#X_test['Age'].replace(np.NaN,25,inplace=True)

#Identify X - Segregate X
X_test['Sex'].replace('male',0, inplace=True)
X_test['Sex'].replace('female',1, inplace=True)
X_test.dropna(inplace=True)

# Predict the test results
Y_prob_pred = result.predict(X_test)

In [109]:
#print(type(Y_prob_pred))
X_test['Survived'] = Y_prob_pred.apply(lambda x: 0 if x <=0.5 else 1)
X_test['PassengerId'] = test_df['PassengerId']

In [107]:
test_df.shape
X_test.shape

(418, 4)

In [110]:
X_test.head()
finalResult = X_test[['PassengerId', 'Survived']]

In [113]:
finalResult.to_csv("result.csv", index=False)